In [ ]:
#this creates a csv file 'universityList.csv' containing name, key and link for all university.
# Note to self
# ------- Do not Execute this cell -----------

getUniversityList()

In [8]:
import requests
import sys
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import urllib.request
import pandas as pd
import numpy as np
import time

In [9]:
def open_url_2_soup(url):
    """
    open webpage and convert to beautiful soup object
    returns: page_soup(soup object)
    params: url(str) - url of the page
    
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
    page = requests.get(url,headers = headers,verify = False, allow_redirects = False)
    if(page.status_code == 200):
        page_soup = soup(page.content,'lxml')
        return page_soup
    else: 
        print("error not found")
        raise HTTPError()

In [10]:
def getUniversityLookup(url):
    """
    get the list of university and it's basic info from ONE page
    returns : univ_df - dataframe containing name,key and url of universities one page at a time
    params: url - url of the page
    
    """
    univ_name = []
    univ_link = []
    univ_soup = open_url_2_soup(url)
    univ_detail_tag = univ_soup.findAll("div",{"class":"col-sm-9 col-xs-12"})
    for tag in univ_detail_tag:
        a = tag.find("a",href=True)
        univ_name.append(a.text)
        univ_link.append("https://yocket.in"+a['href'])
    #data cleaning 
    univ_df = pd.DataFrame({"name":np.asarray(univ_name),"href": np.asarray(univ_link)})
    print(univ_df['name'].str.split('(',n = 1, expand = True))
    univ_df[['name','keys']] = univ_df['name'].str.split('(',n = 1, expand = True)
    univ_df['keys'] = univ_df['keys'].apply(lambda x: x[:-1])
    return univ_df

In [11]:
def getUniversityList():
    """
    creates a csv file of all university detail
    returns: UniversityList_df - dataframe containg the basic details of top 500 engineering schools
    params: None
    
    """
    UniversityList_df = pd.DataFrame()
    count = 1
    while(count <= 16):
        url = "https://yocket.in/universities?page=" + str(count)
        temp_df = getUniversityLookup(url)
#         print(temp_df)
        UniversityList_df = UniversityList_df.append(temp_df,ignore_index=True)
#         print(UniversityList_df)
        time.sleep(5) 
        count += 1
    UniversityList_df.to_csv('universityList.csv', encoding='utf-8')
    return UniversityList_df


In [12]:
def extractUnivDetails(tag,tag_no):
    """
    returns : univ_dict(dictionary) - dictionary containing university(single) detail
    params: tag(soup element) - the div containing the element
            tag_no(int) - number of div
    """
    univ_dict = {}
    if(tag_no in [1,2,8,9]):
        return(tag.h3.small.text,(tag.br.previous_sibling)[1:])
#     elif(tag_no ==3):
#         return (tag.h3.findAll("small")[1].text,tag.h3.find("small").previous_sibling)
    elif(tag_no in [3,4]):
        try:
            name = tag.h3.findAll("small")[1].text
            data = (tag.h3.find("br").previous_sibling[2:])
        except:
            return("average living expense","NA")
        else:
            return (tag.h3.findAll("small")[1].text,(tag.h3.find("small").previous_sibling)[2:])
    elif(tag_no == 6):
        try:
            a = tag.findAll("a",href=True)[0]
        except:
            return("student_link",None)
        else:
            return("student_link",("https://yocket.in"+a['href'])[:-1])


In [13]:

def getUniversityDetail(url):
    """
    get the details of a single university
    params: url(string) - url of the site
    returns : univ_detail(dict) - dictionary containing details of a single university
    
    """
    univ_detail = {}
    try:
        page_soup = open_url_2_soup(url)
        univ_detail['name'] = page_soup.findAll("h1")[0].text
        univ_tag = page_soup.findAll("div",{"class":"col-sm-3 col-xs-6"})

        tag_no = 1;
        for tag in univ_tag:
            if(tag_no in [1,2,3,4,6,8,9]):
                name,data = extractUnivDetails(tag,tag_no)
    #             print(name,data)
                univ_detail[name] = data
            tag_no+= 1
        try:
            univ_detail['admitList'] = univ_detail['student_link']+"2"
            univ_detail['rejectList'] = univ_detail['student_link']+"3"
            univ_detail['appliedList'] = univ_detail['student_link']+"1"
            del univ_detail['student_link']
        except:
            univ_detail['admitList'] = None
            univ_detail['rejectList'] = None
            univ_detail['appliedList'] = None
            del univ_detail['student_link']
    except: 
        print("Exception raised")
    else:
        return(univ_detail)
# getUniversityDetail("https://yocket.in/university-reviews/california-state-university-chico-3145/engineering")

In [18]:
def univDetail_to_csv():
    """
    generate csv file containing university details
    params : None
    returns : None
    """
    univ_list = pd.read_csv('universityList.csv')
    universityDetail = pd.DataFrame()
    counter = 401
    while(counter <= 477):
        link = univ_list.iloc[counter]['href']
        print(link)
        dict_temp = getUniversityDetail(link)
        universityDetail = universityDetail.append(dict_temp,ignore_index = True)
        print(counter)
        counter += 1
        time.sleep(10)
#     print(universityDetail)
    universityDetail.to_csv('universityDetailList.csv', encoding='utf-8',header = False, mode = 'a')

In [19]:
#create the cab file of university list with all it's parameter
#  Note to self
# -----------DO NOT EXECUTE THIS CELL --------------
univDetail_to_csv()   

https://yocket.in/university-reviews/saint-xavier-university-illinois-orland-park-2773/engineering


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


401
https://yocket.in/university-reviews/wesleyan-university-2857/engineering


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


402
https://yocket.in/university-reviews/christopher-newport-university-2689/engineering


...

475
https://yocket.in/university-reviews/rhodes-college-3332/engineering


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


476
https://yocket.in/university-reviews/bryn-mawr-college-3314/engineering


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


477


In [45]:
def getStdList(page_soup,admit_status):
    grading = page_soup.findAll("div",{"class" : "col-sm-6"})
#     student_df = pd.DataFrame()
    for tag in grading:
        print(tag)
        
#         std_data = {}
#         std_data['admit/reject'] = admit_status
# #         std_data['Name'] = tag.find("a").text #Tag for name of student
# #         std_data['CourseName'] = tag.find("small").text #Tag for course
        heading = tag.find("strong").text
        data = tag.findNext("br").next_sibling
#         if (heading == "GRE"):
#             std_data['GRE'] = data.string

#         elif(heading == "TOEFL" or heading == "IELTS" or heading == "ENG TEST"):
#             std_data['ENG_TEST'] = data.string

#         elif("UNDERGRAD"):
#             std_data['CGPA']= data.string

#         elif("WORK EX"):
#             std_data['WorkExp'] = data.string
#         print(std_data)
#         break
#     return std_data


In [ ]:
def getStudentDetailList(url):
    page_sounter = 1
    url = url + "?page=" + str(page+counter)
    while(True)
        try:
            page_soup = open_url_2_soup(url)
        except:
            page_counter = 1
            break
        else:
            name_tag = page_soup.findAll("input" , {"id" : "users-view-search-universities"})[0]
            name = name_tag['value']
            
            page_sounter += 1
        

In [76]:
u = "https://yocket.in/university-reviews/california-state-university-chico-3145/engineering"
page_soup = open_url_2_soup(u)
print(page_soup)
# getStdList(page_soup,'admit')  
#name
# print(data['value']) #name


/home/rbl/anaconda3/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


error not found


NameError: name 'HTTPError' is not defined